In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

pd.set_option('display.max_columns', None)

In [2]:
test_file = 'Input_Data/test.csv'
train_file = 'Input_Data/train.csv'

In [3]:
test_data = pd.read_csv(test_file)
test_data.head()

,id,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,pain,peristalsis,abdominal_distention,nasogastric_tube,nasogastric_reflux,nasogastric_reflux_ph,rectal_exam_feces,abdomen,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data
0,1235,no,adult,534053,38.6,40.0,20.0,normal,normal,normal_pink,less_3_sec,mild_pain,hypomotile,slight,none,none,7.0,normal,distend_small,42.0,7.5,clear,2.3,no,0,0,0,no
1,1236,yes,adult,528469,38.2,112.0,48.0,cool,reduced,bright_pink,more_3_sec,depressed,hypomotile,moderate,slight,none,3.5,decreased,distend_small,44.0,6.0,serosanguious,2.6,no,2208,0,0,yes
2,1237,yes,adult,528178,37.7,66.0,12.0,cool,normal,bright_red,less_3_sec,mild_pain,hypomotile,slight,slight,none,3.0,normal,distend_small,31.5,6.0,cloudy,1.6,yes,2205,0,0,yes
3,1238,no,adult,534784,37.1,88.0,20.0,cool,reduced,pale_cyanotic,less_3_sec,depressed,absent,severe,slight,more_1_liter,2.0,absent,distend_large,75.0,81.0,None,1.0,yes,1400,0,0,no
4,1239,yes,adult,529840,38.3,50.0,12.0,None,normal,bright_pink,less_3_sec,mild_pain,absent,slight,slight,none,3.0,decreased,distend_small,37.0,6.8,cloudy,2.6,yes,2208,0,0,yes


In [4]:
train_data = pd.read_csv(train_file)
train_data.head()

,id,surgery,age,hospital_number,rectal_temp,pulse,respiratory_rate,temp_of_extremities,peripheral_pulse,mucous_membrane,capillary_refill_time,pain,peristalsis,abdominal_distention,nasogastric_tube,nasogastric_reflux,nasogastric_reflux_ph,rectal_exam_feces,abdomen,packed_cell_volume,total_protein,abdomo_appearance,abdomo_protein,surgical_lesion,lesion_1,lesion_2,lesion_3,cp_data,outcome
0,0,yes,adult,530001,38.1,132.0,24.0,cool,reduced,dark_cyanotic,more_3_sec,depressed,absent,slight,slight,less_1_liter,6.5,decreased,distend_small,57.0,8.5,serosanguious,3.4,yes,2209,0,0,no,died
1,1,yes,adult,533836,37.5,88.0,12.0,cool,normal,pale_cyanotic,more_3_sec,mild_pain,absent,moderate,none,more_1_liter,2.0,absent,distend_small,33.0,64.0,serosanguious,2.0,yes,2208,0,0,no,euthanized
2,2,yes,adult,529812,38.3,120.0,28.0,cool,reduced,pale_pink,less_3_sec,extreme_pain,hypomotile,moderate,slight,none,3.5,None,distend_large,37.0,6.4,serosanguious,3.4,yes,5124,0,0,no,lived
3,3,yes,adult,5262541,37.1,72.0,30.0,cold,reduced,pale_pink,more_3_sec,mild_pain,hypomotile,moderate,slight,more_1_liter,2.0,decreased,distend_small,53.0,7.0,cloudy,3.9,yes,2208,0,0,yes,lived
4,4,no,adult,5299629,38.0,52.0,48.0,normal,normal,normal_pink,less_3_sec,alert,hypomotile,none,slight,less_1_liter,7.0,normal,normal,47.0,7.3,cloudy,2.6,no,0,0,0,yes,lived


In [5]:
# from sklearn.tree import DecisionTreeClassifier -- This needs categorical data changed to numerical

# model = DecisionTreeClassifier()
# model.fit(X_train,y_train)

In [6]:
##Adding missing data
##EA was done as part of another notebook.  Will try and keep the rest of the notebooks clean.
for column in train_data.select_dtypes(include=["object"]):
    train_data[column]=train_data[column].fillna("no_measure_recorded")
    
for column in test_data.select_dtypes(include=["object"]):
    test_data[column]=test_data[column].fillna("no_measure_recorded")
    
for column in train_data.select_dtypes(include=["int64"]):
    train_data[column]=train_data[column].fillna(0)
    
for column in test_data.select_dtypes(include=["int64"]):
    test_data[column]=test_data[column].fillna(0)

for column in train_data.select_dtypes(include=["float64"]):
    train_data[column]=train_data[column].fillna(0)
    
for column in test_data.select_dtypes(include=["float64"]):
    test_data[column]=test_data[column].fillna(0)

In [7]:
X_train = train_data.drop(columns=["outcome","id","hospital_number"])
y_train = train_data["outcome"]
X_test = test_data

In [8]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

categorical_cols = [cname for cname in X_train.columns if
                   X_train[cname].dtype == object]

categorical_cols

['surgery',
 'age',
 'temp_of_extremities',
 'peripheral_pulse',
 'mucous_membrane',
 'capillary_refill_time',
 'pain',
 'peristalsis',
 'abdominal_distention',
 'nasogastric_tube',
 'nasogastric_reflux',
 'rectal_exam_feces',
 'abdomen',
 'abdomo_appearance',
 'surgical_lesion',
 'cp_data']

In [9]:
categorical_cols_indices = [X_train.columns.get_loc(cname) for cname in categorical_cols]

In [10]:
## ValueError: Found unknown categories ['moderate'] in column 6 during transform -- data in the test wasn't present in the training
## addedhandle_unknown='ignore' to OneHotEncoder
##after adding ignore  ValueError: Specifying the columns using strings is only supported for pandas DataFrames
## Got the indicies of the cat columns
##Problem might have been the need to restart kernal.  Will make a habit to restart after every error
preprocessor = ColumnTransformer(
transformers=[
    ('cat',OneHotEncoder(drop='first', handle_unknown='ignore'),categorical_cols_indices)
],remainder='passthrough')

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

# /opt/conda/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:202: UserWarning: Found unknown categories in columns [6] during transform. These unknown categories will be encoded as all zeros
#   warnings.warn(

C:\ProgramData\anaconda3\Lib\site-packages\sklearn\preprocessing\_encoders.py:227: UserWarning: Found unknown categories in columns [6] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


In [11]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [12]:
y_pred = model.predict(X_test)

In [13]:
test_data["outcome"] = y_pred

In [14]:
output = test_data[['id', 'outcome']]
output.to_csv('outputII.csv', index=False)
#Score: 0.74390  Rank upper 400's  Not Too Shabby for my first try and brute force method.
#ScoreII: 0.76829